In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

import matplotlib as plt

In [2]:
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

In [4]:
#set the location of the directory with data
directory = '/home/shenbaga/data-files/House_Prices/'

In [5]:
train = pd.read_csv(directory + 'train.csv')

In [6]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
test = pd.read_csv(directory + 'test.csv')

In [8]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [9]:
# drop a few columns that have very low data fill rate
#poor fill rate in train: Alley, FireplaceQu, PoolQC, Fence, MiscFeature
#poor fill rate in test: Alley, FireplaceQu, PoolQC, Fence, MiscFeature

In [10]:
train.drop(['Alley','FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'],axis=1,inplace=True)

In [11]:
test.drop(['Alley','FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'],axis=1,inplace=True)
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,120,0,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,...,36,0,0,0,0,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,...,34,0,0,0,0,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,...,36,0,0,0,0,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,...,82,0,0,144,0,0,1,2010,WD,Normal


In [12]:
#concatenate the two dfs and encode object colums

In [13]:
ntrain = len(train)
ntest = len(test)

ntrain, ntest

(1460, 1459)

In [14]:
concat_df = pd.concat([train, test],axis = 0)
concat_df.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,3,1Fam,TA,No,706.0,0.0,GLQ,...,WD,0,Pave,8,856.0,AllPub,0,2003,2003,2008
1,1262,0,0,3,1Fam,TA,Gd,978.0,0.0,ALQ,...,WD,0,Pave,6,1262.0,AllPub,298,1976,1976,2007
2,920,866,0,3,1Fam,TA,Mn,486.0,0.0,GLQ,...,WD,0,Pave,6,920.0,AllPub,0,2001,2002,2008
3,961,756,0,3,1Fam,Gd,No,216.0,0.0,ALQ,...,WD,0,Pave,7,756.0,AllPub,0,1915,1970,2006
4,1145,1053,0,4,1Fam,TA,Av,655.0,0.0,GLQ,...,WD,0,Pave,9,1145.0,AllPub,192,2000,2000,2008


In [15]:
#encode object cols
for feature in concat_df.columns:
    if concat_df[feature].dtype == 'object':
        concat_df[feature] = pd.Categorical(concat_df[feature]).codes

In [16]:
concat_df.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,3,0,3,3,706.0,0.0,2,...,8,0,1,8,856.0,0,0,2003,2003,2008
1,1262,0,0,3,0,3,1,978.0,0.0,0,...,8,0,1,6,1262.0,0,298,1976,1976,2007
2,920,866,0,3,0,3,2,486.0,0.0,2,...,8,0,1,6,920.0,0,0,2001,2002,2008
3,961,756,0,3,0,1,3,216.0,0.0,0,...,8,0,1,7,756.0,0,0,1915,1970,2006
4,1145,1053,0,4,0,3,0,655.0,0.0,2,...,8,0,1,9,1145.0,0,192,2000,2000,2008


In [17]:
#split the concat back to train and test
final_train = concat_df[:ntrain]
final_test = concat_df[ntrain:]

In [18]:
len(final_train), len(final_test)

(1460, 1459)

In [19]:
final_test.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,896,0,0,2,0,3,3,468.0,144.0,4,...,8,120,1,5,882.0,0,140,1961,1961,2010
1,1329,0,0,3,0,3,3,923.0,0.0,0,...,8,0,1,6,1329.0,0,393,1958,1958,2010
2,928,701,0,3,0,3,3,791.0,0.0,2,...,8,0,1,6,928.0,0,212,1997,1998,2010
3,926,678,0,3,0,3,3,602.0,0.0,2,...,8,0,1,7,926.0,0,360,1998,1998,2010
4,1280,0,0,2,4,3,3,263.0,0.0,0,...,8,144,1,5,1280.0,0,0,1992,1992,2010


In [20]:
final_train.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,3,0,3,3,706.0,0.0,2,...,8,0,1,8,856.0,0,0,2003,2003,2008
1,1262,0,0,3,0,3,1,978.0,0.0,0,...,8,0,1,6,1262.0,0,298,1976,1976,2007
2,920,866,0,3,0,3,2,486.0,0.0,2,...,8,0,1,6,920.0,0,0,2001,2002,2008
3,961,756,0,3,0,1,3,216.0,0.0,0,...,8,0,1,7,756.0,0,0,1915,1970,2006
4,1145,1053,0,4,0,3,0,655.0,0.0,2,...,8,0,1,9,1145.0,0,192,2000,2000,2008


In [21]:
features = list(concat_df.columns.values)

In [22]:
type(features)

list

In [26]:
features

['1stFlrSF',
 '2ndFlrSF',
 '3SsnPorch',
 'BedroomAbvGr',
 'BldgType',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtFinType1',
 'BsmtFinType2',
 'BsmtFullBath',
 'BsmtHalfBath',
 'BsmtQual',
 'BsmtUnfSF',
 'CentralAir',
 'Condition1',
 'Condition2',
 'Electrical',
 'EnclosedPorch',
 'ExterCond',
 'ExterQual',
 'Exterior1st',
 'Exterior2nd',
 'Fireplaces',
 'Foundation',
 'FullBath',
 'Functional',
 'GarageArea',
 'GarageCars',
 'GarageCond',
 'GarageFinish',
 'GarageQual',
 'GarageType',
 'GarageYrBlt',
 'GrLivArea',
 'HalfBath',
 'Heating',
 'HeatingQC',
 'HouseStyle',
 'Id',
 'KitchenAbvGr',
 'KitchenQual',
 'LandContour',
 'LandSlope',
 'LotArea',
 'LotConfig',
 'LotFrontage',
 'LotShape',
 'LowQualFinSF',
 'MSSubClass',
 'MSZoning',
 'MasVnrArea',
 'MasVnrType',
 'MiscVal',
 'MoSold',
 'Neighborhood',
 'OpenPorchSF',
 'OverallCond',
 'OverallQual',
 'PavedDrive',
 'PoolArea',
 'RoofMatl',
 'RoofStyle',
 'SaleCondition',
 'SaleType',
 'ScreenPorch',
 'Street',
 '

In [27]:
features.remove('Id')

In [28]:
y_train = final_train['SalePrice']

In [29]:
#step 1. use Sklearn XGB and Grid Search for param tuning

In [30]:
sklearn_xgb_fixed_params = {'seed':0,
                           'objective': 'reg:linear',
                            'n_estimators': 1000,
                            'learning_rate':0.1
                           }

sklearn_xgb_params_to_be_tuned = {'subsample': [0.6,0.8,1.0],
                                  'colsample_bytree': [0.6,0.8,1.0],
                                  'max_depth': [3,5,7],
                                  'min_child_weight': [1,3,5]
                                 }

In [31]:
sklearn_xgb_model = XGBRegressor(**sklearn_xgb_fixed_params)

In [32]:
#now grid search using the params to be tuned

In [33]:
from time import time

In [36]:
from datetime import datetime

In [35]:
from sklearn.grid_search import GridSearchCV

/home/shenbaga/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/shenbaga/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [37]:
print datetime.now()

sklearn_xgb_model_Grid_Searched = GridSearchCV(sklearn_xgb_model,
                                               sklearn_xgb_params_to_be_tuned,
                                               cv = 5)

print datetime.now()

2016-12-15 16:45:18.084385
2016-12-15 16:45:18.085208


In [41]:
#fit the data in this model:
print datetime.now()

sklearn_xgb_model_Grid_Searched.fit(final_train[features], y_train) #note that the input is not a DMatrix

print datetime.now()

2016-12-15 16:47:43.764174
2016-12-15 17:15:01.225688


In [42]:
sklearn_xgb_model_Grid_Searched.best_params_

{'colsample_bytree': 1.0,
 'max_depth': 3,
 'min_child_weight': 3,
 'subsample': 0.8}

In [43]:
sklearn_xgb_model_Grid_Searched.best_score_

0.8983335642267948

#so the best vals for the parameters are:
{'colsample_bytree': 1.0,
 'max_depth': 3,
 'min_child_weight': 3,
 'subsample': 0.8}

In [50]:
from sklearn.metrics import mean_squared_error

In [51]:
#train rmse:
np.sqrt(mean_squared_error(y_true=y_train, 
                           y_pred=sklearn_xgb_model_Grid_Searched.predict(final_train[features])))

3387.517192125043

In [52]:
np.mean(y_train), np.std(y_train)

(180921.19589041095, 79415.29188606751)

In [54]:
#use xgb.cv to find n rounds param
tuned_params = {
                'seed':0,
                'objective': 'reg:linear',
                'eta':0.1,
                'colsample_bytree': 1.0,
                'max_depth': 3,
                'min_child_weight': 3,
                'subsample': 0.8
                }

In [55]:
dfinaltrain = xgb.DMatrix(final_train[features], y_train)
dfinaltest = xgb.DMatrix(final_test[features])

In [56]:
xgb_cv = xgb.cv(dtrain=dfinaltrain,
                params=tuned_params,
                verbose_eval=50,
                early_stopping_rounds=50,
                num_boost_round=10000,
                nfold=5)

[0]	train-rmse:178967+1855.99	test-rmse:178862+7915.95
[50]	train-rmse:19158.6+424.286	test-rmse:28461.9+4309.21
[100]	train-rmse:14930.2+468.96	test-rmse:27277.5+4612.51
[150]	train-rmse:12684.2+471.666	test-rmse:26748.5+4419.67
[200]	train-rmse:11079.7+412.589	test-rmse:26326.8+4259.98
[250]	train-rmse:9814.12+390.711	test-rmse:26218.8+4309.51
[300]	train-rmse:8764.9+332.339	test-rmse:26041.4+4314.06
[350]	train-rmse:7822.93+318.141	test-rmse:25999.2+4301.99
[400]	train-rmse:7043.17+284.738	test-rmse:25973.2+4292.63
[450]	train-rmse:6365.49+230.34	test-rmse:25935.1+4325.34
[500]	train-rmse:5773.19+217.881	test-rmse:25936.1+4291.9


In [57]:
xgb_cv.tail(5)

,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
483,25927.283594,4301.199154,5965.859277,223.555608
484,25925.894922,4302.392532,5954.354785,223.624338
485,25923.199219,4300.373354,5943.408789,223.104568
486,25921.102344,4301.089585,5931.834570,225.113021
487,25919.969531,4298.863783,5918.802539,222.691749


In [58]:
xgb_cv_trained_model = xgb.train(dtrain=dfinaltrain,
                                 params=tuned_params,
                                 num_boost_round=xgb_cv.shape[0]
                                )

In [59]:
#train rmse for xgb:
np.sqrt(mean_squared_error(y_true=y_train, 
                           y_pred=xgb_cv_trained_model.predict(xgb.DMatrix(final_train[features]))))

6978.7334345340687

In [60]:
#again, I am not sure what is happening. the rmse of the xgb cv model is higher than that of the 
#sklearn model before xgb cv

In [61]:
#test preds for submission from both these models:

#for the sklearn model
submission = pd.read_csv(directory + 'sample_submission.csv')
submission.iloc[:, 1] = sklearn_xgb_model_Grid_Searched.predict(final_test[features])
submission.to_csv('Sklearn_XGB_Tuned_Test.csv', index=None)

#for the xgb cv model
submission = pd.read_csv(directory + 'sample_submission.csv')
submission.iloc[:, 1] = xgb_cv_trained_model.predict(dfinaltest)
submission.to_csv('Sklearn_XGB_with_xgb_cv_Test.csv', index=None)

In [62]:
#sklearn model - Your submission scored 0.13465
#xgb cv model - 0.13433,

In [63]:
#still not convinced abt the best score function; hypothesis is that least shoould be the best and not the
#largest as returned by the function

In [64]:
scores = sklearn_xgb_model_Grid_Searched.grid_scores_

In [65]:
type(scores)

list

In [66]:
scores[1]

mean: 0.88623, std: 0.02889, params: {'subsample': 0.8, 'colsample_bytree': 0.6, 'max_depth': 3, 'min_child_weight': 1}

In [67]:
a = scores[1]
type(a)

sklearn.grid_search._CVScoreTuple

In [68]:
a[0], a[1], a[2]

({'colsample_bytree': 0.6,
  'max_depth': 3,
  'min_child_weight': 1,
  'subsample': 0.8},
 0.8862341716882953,
 array([ 0.91451502,  0.83646093,  0.89456637,  0.91206307,  0.87356547]))

In [70]:
minm = 1
min_index = 0
for i in range(len(scores)):
    x = scores[i][1]
    if x < minm:
        minm = x
        min_index = i
        
print minm, min_index, scores[min_index]

0.869008278684 51 mean: 0.86901, std: 0.05081, params: {'subsample': 0.6, 'colsample_bytree': 0.8, 'max_depth': 7, 'min_child_weight': 5}


In [71]:
#so the other end of params is:
#{'subsample': 0.6, 'colsample_bytree': 0.8, 'max_depth': 7, 'min_child_weight': 5}

In [72]:
tuned_params2 = {
                'seed':0,
                'objective': 'reg:linear',
                'n_estimators': 1000,
                'learning_rate':0.1,
                'subsample': 0.6,
                'colsample_bytree': 0.8,
                'max_depth': 7,
                'min_child_weight': 5
                }

In [73]:
#build sklearn model with these params
sklearn_xgb_model_Grid_Searched2 = XGBRegressor(**tuned_params2)

In [74]:
sklearn_xgb_model_Grid_Searched2.fit(final_train[features], y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=5, missing=None, n_estimators=1000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.6)

In [75]:
np.sqrt(mean_squared_error(y_true=y_train, 
                           y_pred=sklearn_xgb_model_Grid_Searched2.predict(final_train[features])))

126.93169996388578

In [76]:
#shocking to see the above RMSE.!! note that the RMSE of the best params in sklearn was 3387.XXX

In [77]:
#do test predictions and see the submission score

submission = pd.read_csv(directory + 'sample_submission.csv')
submission.iloc[:, 1] = sklearn_xgb_model_Grid_Searched2.predict(final_test[features])
submission.to_csv('Sklearn_XGB_Tuned2_Test.csv', index=None)

In [78]:
#above scored 0.13582, whereas the best params scored 0.13465
#so not sure if anything has improved

In [79]:
#continue doing the xgb cv part

In [80]:
#params from the above sklearn part
tuned_params3 = {
                'seed':0,
                'objective': 'reg:linear',
                'eta':0.1,
                'colsample_bytree': 0.8,
                'max_depth': 7,
                'min_child_weight': 5,
                'subsample': 0.6
                }

In [81]:
xgb_cv2 = xgb.cv(dtrain=dfinaltrain,
                params=tuned_params3,
                verbose_eval=50,
                early_stopping_rounds=50,
                num_boost_round=10000,
                nfold=5)

[0]	train-rmse:178975+1932.56	test-rmse:178736+7710.21
[50]	train-rmse:13818+335.639	test-rmse:27774.8+5670.44
[100]	train-rmse:9599.39+497.928	test-rmse:27813.4+6650.58


In [82]:
xgb_cv2.tail(5)

,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
61,27721.887891,5992.000272,12557.205469,464.741257
62,27727.255078,6030.526682,12469.097852,484.861562
63,27650.719531,6038.832860,12400.795313,497.123215
64,27629.246875,6067.869222,12291.266016,480.616223
65,27629.131250,6123.655603,12220.783594,464.179382


In [83]:
xgb_cv_trained_model2 = xgb.train(dtrain=dfinaltrain,
                                 params=tuned_params3,
                                 num_boost_round=xgb_cv2.shape[0]
                                )

In [84]:
np.sqrt(mean_squared_error(y_true=y_train, 
                           y_pred=xgb_cv_trained_model2.predict(xgb.DMatrix(final_train[features]))))

12548.447035234985

In [85]:
#again, the rmse above is way too higher than all others so far

In [86]:
#for the xgb cv 2 model
submission = pd.read_csv(directory + 'sample_submission.csv')
submission.iloc[:, 1] = xgb_cv_trained_model2.predict(dfinaltest)
submission.to_csv('Sklearn_XGB_with_xgb_cv2_Test.csv', index=None)

In [87]:
#above submission scored 0.13447

in summary, the scores so far (in order) are:
#sklearn model -0.13465
#xgb cv model - 0.13433
#sklearn model2 - 0.13582
#xgb cv model2 - 0.13447

In [88]:
#do the kfold cross val

In [89]:
import gc

In [90]:
from sklearn.cross_validation import KFold

In [91]:
#params for kfold
params_for_kfold = {
                    'seed':0,
                    'objective': 'reg:linear',
                    'learning_rate':0.1,
                    'colsample_bytree': 1.0,
                    'max_depth': 3,
                    'min_child_weight': 3,
                    'subsample': 0.8
                    }

In [92]:
best_nrounds = 20000

allpredictions = pd.DataFrame()

kfolds = 10  # 10 folds is better!

if kfolds > 1:
    
    kf = KFold(final_train.shape[0], n_folds=kfolds)
    
    for i, (train_index, test_index) in enumerate(kf):
        
        dtest = xgb.DMatrix(final_test[features])
        
        print('Fold {0}'.format(i + 1))
        
        X_train, X_val = final_train.iloc[train_index], final_train.iloc[test_index]
        
        dtrain = xgb.DMatrix(X_train[features], label=X_train.SalePrice)
        
        dvalid = xgb.DMatrix(X_val[features],label=X_val.SalePrice)
        
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

        gbdt = xgb.train(params_for_kfold, dtrain, best_nrounds, watchlist,
                         verbose_eval=50,
                         early_stopping_rounds=25)
        
        del dtrain
        del dvalid
        
        gc.collect()
        
        allpredictions['p' + str(i)] = \
            gbdt.predict(dtest, ntree_limit=gbdt.best_ntree_limit)
        
        del dtest
        del gbdt
        gc.collect()

Fold 1
[0]	train-rmse:179964	eval-rmse:169043
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 25 rounds.
[50]	train-rmse:19313.8	eval-rmse:21803.3
[100]	train-rmse:15404.5	eval-rmse:20666.4
[150]	train-rmse:13217.9	eval-rmse:19961.1
[200]	train-rmse:11650.5	eval-rmse:19866.3
[250]	train-rmse:10457.2	eval-rmse:19498.7
Stopping. Best iteration:
[249]	train-rmse:10478.1	eval-rmse:19471.9

Fold 2
[0]	train-rmse:178334	eval-rmse:186237
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 25 rounds.
[50]	train-rmse:18980.1	eval-rmse:27268.7
[100]	train-rmse:15243.9	eval-rmse:25678.7
[150]	train-rmse:13185.9	eval-rmse:24704.6
Stopping. Best iteration:
[144]	train-rmse:13299.6	eval-rmse:24596.5

Fold 3
[0]	train-rmse:178911	eval-rmse:180096
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will tra

In [93]:
allpredictions.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
0,127536.757812,128838.031250,123163.562500,127308.921875,121290.820312,123225.109375,124203.085938,125067.250000,128004.843750,125419.093750
1,158325.343750,161572.656250,162072.296875,154392.140625,156649.609375,157050.421875,164128.312500,164468.406250,161747.015625,160768.234375
2,175606.250000,174511.890625,172930.062500,169796.859375,183607.609375,172831.859375,181433.375000,181788.515625,184877.500000,175596.843750
3,196732.593750,189432.750000,192209.406250,175048.703125,184781.234375,184545.453125,193408.062500,200285.484375,185971.359375,190618.937500
4,191624.375000,200752.484375,187043.640625,209825.921875,203819.468750,206147.312500,184400.468750,201183.484375,204631.859375,207581.750000


In [94]:
#for submission

submission = pd.read_csv(directory + 'sample_submission.csv')
submission.iloc[:, 1] = allpredictions.mean(axis=1).values
submission.to_csv('xgb_10fold_with_tuning.csv', index=None)

In [95]:
#this scored 0.12990,

in summary, the scores so far (in order) are:
#sklearn model -0.13465
#xgb cv model - 0.13433
#sklearn model2 - 0.13582
#xgb cv model2 - 0.13447
#10 fold - 0.12990

In [96]:
#so atleast this makes sense, the 10 folds has a better result that all else